# Steckbrief einer elementaren Funktion

Zuerst wird in der gegebenen Funktion für alle 'c' eine zufällige Konstante aus der Menge {1,2,3} eingesetzt. Daher wird diese zunächst als String erwartet.

In [1]:
from sympy import *
import re
import random
x = Symbol('x', real=True)

f_str = 'log(x)*sin(x)'

### Zufällig ausgewählte Funktion:

In [2]:
f = sympify(re.sub(r"c(?!o)", lambda m: str(random.choice([1,2,3])), f_str), locals={'x': x})
print(f)
f

log(x)*sin(x)


log(x)*sin(x)

In [3]:
from IPython.display import Markdown
from urllib.parse import quote_plus

Markdown(f'[Wolfram Alpha](https://www.wolframalpha.com/input?i={quote_plus(str(f))})')

[Wolfram Alpha](https://www.wolframalpha.com/input?i=log%28x%29%2Asin%28x%29)

### Definitionsbereich

In [4]:
domain = calculus.util.continuous_domain(f, x, S.Reals)
print(str(domain))
domain

Interval.open(0, oo)


Interval.open(0, oo)

### Wertebereich

**Achtung!** Der Wertebereich kann unter Umständen nicht berechenbar sein 

In [5]:
f_range = None
try:
    f_range = calculus.util.function_range(f, x, S.Reals)
    print(str(f_range))
except (NotImplementedError, ValueError, TypeError):
    print('Wertebereich nicht berechenbar')
f_range

Wertebereich nicht berechenbar


### Unstetigkeitsstellen

In [20]:
limits = {}
try:
    f_singularities = singularities(f, x, domain)
    if f_singularities.is_FiniteSet:
        print(len(f_singularities), ' Unstetigkeitsstelle(n)')
        for s in f_singularities:
            left_limit = limit(f, x, s, '-')
            right_limit = limit(f, x, s, '+')
            limits[s] = (left_limit, right_limit)
            f_x = f.subs(x, s)
            if left_limit.is_real and left_limit == right_limit:
                print('Hebbare Lücke: ', s)
            elif left_limit.is_real and right_limit.is_real and left_limit != right_limit:
                print('Sprungstelle: ', s)
            elif any([not l for l in [left_limit, right_limit]]):
                print('Wesentliche Singularität: ', s)
            elif all([not l.as_real_imag()[1] for l in [left_limit, right_limit]]):
                print('Polstelle: ', s)
            if f_x == left_limit:
                print('(linksseitig)')
            elif f_x == right_limit:
                print('(rechtsseitig)')
    else:
        print('Unendlich Unstetigkeitsstellen')
        print(f_singularities)
        f_singularities = None
except NotImplementedError:
    print('Unstetigkeitsstellen nicht berechenbar')

0  Unstetigkeitsstelle(n)


### Grenzwerte

In [7]:
for p in FiniteSet(-oo, oo):
    try:
        l = limit(f, x, p)
        limits[p] = (l)
    except (ValueError, NotImplementedError):
        print('Grenzwert bei ', p, ' nicht berechenbar')

for n, lim in limits.items():
    print('Grenzwert bei ', n, ': ', lim)

Grenzwert bei  0 :  (0, 0)
Grenzwert bei  -oo :  AccumBounds(-oo, oo)
Grenzwert bei  oo :  AccumBounds(-oo, oo)


### Asymptoten

In [8]:
# Calculation method taken from https://en.wikipedia.org/wiki/Asymptote#Elementary_methods_for_identifying_asymptotes
# Also mentioned in https://encyclopediaofmath.org/index.php?title=Asymptote
def asymptotes():
    a = set()
    if f.is_polynomial(x):
        return a
    for lim in [oo, -oo]:
        try:
            m = limit(f/x, x, lim)
            if m.is_real:
                n = limit(f-m*x, x, lim)
                if n.is_real and n.is_number:
                    a.add(m*x+n)
        except NotImplementedError:
            print('Asymptote nicht berechenbar')
    return a

a = asymptotes()
print('Asymptoten' if a else 'Keine Asymptoten')
a

Keine Asymptoten


set()

### Periodizität

In [9]:
p = periodicity(f, x)
print('Periodisch' if p else 'Nicht periodisch')
p

Nicht periodisch


### y-Achsenschnitt

In [10]:
y_intercept = f.subs(x, 0)
if y_intercept.is_real:
    print(y_intercept)
else:
    print('Kein y-Achsenschnittpunkt')

Kein y-Achsenschnittpunkt


### Nullstellen

Eine Funktion kann eine endliche oder unendliche Menge an Nullstellen haben, daher muss hier differenziert werden.

Die Funktion `solveset()` liefert die genaue Menge der Nullstellen algebraisch definiert. Diese ist jedoch für spätere Filterung schlechter geeignet.

In [11]:
all_zeros = solveset(f, x, S.Reals)
infinite_zeros = not all_zeros.is_FiniteSet

if infinite_zeros:
    if isinstance(all_zeros, ConditionSet):
        print('Menge der Nullstellen nicht berechnet')
    else:
        print(srepr(all_zeros))
        print('Menge der Nullstellen ist unendlich')
else:
    print('Menge der Nullstellen ist endlich')
all_zeros

Union(FiniteSet(Integer(1)), ImageSet(Lambda(Tuple(Dummy('n', dummy_index=8366906)), Mul(Integer(2), Dummy('n', dummy_index=8366906), pi)), Integers), ImageSet(Lambda(Tuple(Dummy('n', dummy_index=8366901)), Add(Mul(Integer(2), Dummy('n', dummy_index=8366901), pi), pi)), Integers))
Menge der Nullstellen ist unendlich


Union({1}, ImageSet(Lambda(_n, 2*_n*pi + pi), Integers), ImageSet(Lambda(_n, 2*_n*pi), Integers))

Mit Hilfe von `solve()` wird immer eine endliche Menge zurückgegeben, die allerdings im Falle unendlicher Nullstellen nicht vollständig ist (z.B. bei $sin(x)$):

In [12]:
finite_zeros = None
try:
    finite_zeros = solve(f, x) if infinite_zeros else all_zeros
    if len(finite_zeros) and not any([z.is_real for z in finite_zeros]):
        print('Keine reellwertigen Nullstellen')
except NotImplementedError:
    print('Nullstellen nicht berechenbar')
finite_zeros

[0, 1, pi]

### Ableitung

In [13]:
fd = diff(f, x)
fd

log(x)*cos(x) + sin(x)/x

### Extremstellen

In [14]:
fd_zeros = solveset(fd, x, domain)
fdd = diff(fd, x)
extrema = FiniteSet()
if fd_zeros.is_empty:
    print('Keine Extremstellen')
elif fd_zeros.is_FiniteSet:
    for z in fd_zeros:
        val = fdd.subs(x, z)
        if val.is_negative:
            extrema += FiniteSet(z)
            print('Maximum: ', z)
        elif val.is_positive:
            extrema += FiniteSet(z)
            print('Minimum: ', z)
else:
    extrema = None
    print('Extremstellen nicht elementar berechenbar oder unendlich')
    print(fd_zeros)

Extremstellen nicht elementar berechenbar oder unendlich
ConditionSet(x, Eq(x*log(x)*cos(x) + sin(x), 0), Interval.open(0, oo))


### Wendestellen

In [15]:
fdd_zeros = solveset(fdd, x, domain)
inflections = FiniteSet()
if fdd_zeros.is_empty:
    print('Keine Wendepunkte')
elif fdd_zeros.is_FiniteSet:
    fddd = diff(fd, x)
    for z in fdd_zeros:
        val = fddd.subs(x, z)
        if not val.is_zero:
            print('Wendepunkt: ', z)
            inflections += FiniteSet(z)
else:
    inflections = None
    print('Wendepunkte nicht elementar berechenbar oder unendlich')
    print(fdd_zeros)

Wendepunkte nicht elementar berechenbar oder unendlich
ConditionSet(x, Eq(x**2*log(x)*sin(x) - 2*x*cos(x) + sin(x), 0), Interval.open(0, oo))


### Monotonieintervalle

In [16]:
# https://www.massmatics.de/merkzettel/#!144:Monotonie_von_Funktionen
def monotonicity():
    if f_singularities == None or extrema == None:
        print('Monotonieintervalle nicht berechenbar oder unendlich')
        return None
    interval_points = list(f_singularities.union(extrema).union(FiniteSet(-oo, oo)))
    interval_points.sort()
    intervals = [Interval.open(prev, curr) for prev, curr in zip(interval_points, interval_points[1:])]
    for interval in intervals:
        if not interval.is_subset(domain):
            pass
        elif is_strictly_increasing(f, interval, x):
            print('streng monoton steigend in ', interval)
        elif is_strictly_decreasing(f, interval, x):
            print('streng monoton fallend in ', interval)
        elif is_increasing(f, interval, x):
            print('monoton steigend in ', interval)
        elif is_decreasing(f, interval, x):
            print('monoton fallend in ', interval)

monotonicity()

Monotonieintervalle nicht berechenbar oder unendlich


### Konvexitäts-/Konkavitätsbereiche

In [17]:
# https://www.massmatics.de/merkzettel/#!193:Konvexitaet
def convexity():
    if f_singularities == None or inflections == None:
        print('Konvexitäts-/Konkavitätsbereiche nicht berechenbar oder unendlich')
        return None
    interval_points = list(f_singularities.union(inflections).union(FiniteSet(-oo, oo)))
    interval_points.sort()
    intervals = [Interval.open(prev, curr) for prev, curr in zip(interval_points, interval_points[1:])]
    for interval in intervals:
        if is_convex(f, x, domain=interval):
            print('konvex in ', interval)
        elif is_convex(-f, x, domain=interval):
            print('konkav in ', interval)

convexity()

Konvexitäts-/Konkavitätsbereiche nicht berechenbar oder unendlich


### Integral

In [18]:
from sympy.integrals.risch import NonElementaryIntegral
from sympy.integrals.manualintegrate import integral_steps

def calculate_integral():
    integral = integrate(f, x, risch=True)
    print(integral)
    if isinstance(integral, NonElementaryIntegral):
        print('Integral ist nicht elementar')
        return None
    # for arg in preorder_traversal(integral):
    #     if isinstance(arg, Integral):
    #         print('Integral nicht vollständig gelöst')
    #         return integral
    # Manuelle Integration
    steps = integral_steps(f, x)
    rules = set()
    for match in re.finditer('\w+Rule', str(steps)):
        rules.add(match.group())
    print('Integrationsregeln: ', rules)
    return integrate(f, x)

calculate_integral()

Integral(log(x)*sin(x), x)


Integrationsregeln:  {'URule', 'ConstantTimesRule', 'TrigRule', 'CiRule', 'PartsRule', 'AlternativeRule', 'ConstantRule', 'ExpRule'}


-log(x)*cos(x) + Ci(x)

#### Zum Rumprobieren

In [19]:
from scipy import optimize
n = nsolve(f, x, 0, dict=True)
print(n)
if not len(n):
    nf = optimize.fsolve(lambdify(x, f, "numpy"), 0)
    print(nf)

[{x: nan}]
